In [65]:
##Instalación de Pycaret:
##Pipeline de pycaret:

#1. Setup(): Definir el dataset y parámetros
#2. Compare_models(): Comparar multiples modelos y los rankear
#3. Create model(): Crear un modelo específico
#4. Tune_model(): Ajuste automático de hiperparámetros
#5. plot_model(): visualizar el desempeño del modelo
#6. Evaluate_model(): Evaluación del modelo
#7. Predict_model(): Predicciones sobre nuevos datos en el DF
#8. Save_model/load_model(): Guardar y cargar el modelo

In [66]:
##CASO: Predicción sobre si es propenso a caer en dafualt para un credito vehicular
##Objetivo: Construir un modelo de clasificación automático con pycaret para predecir si caera en default en base a sus features

In [67]:
import pandas as pd
from pycaret.classification import *

In [68]:
#1. Cargamos la data
df = pd.read_csv('ds_credito_vehicular.csv')

In [69]:
df.head()

,edad,ingresos_mensuales,estado_civil,historia_credito,nro_creditos_previos,cuota_vs_ingreso,vehiculo_propio,default
0,59,7668,Divorciado,Buena,2,0.64,0,1
1,49,6279,Casado,Buena,2,0.31,1,1
2,35,2722,Soltero,Mala,4,0.41,1,0
3,63,4314,Divorciado,Regular,4,0.57,0,0
4,28,4157,Soltero,Buena,1,0.34,0,0


In [70]:
df['default'].value_counts()

default
0    125
1     25
Name: count, dtype: int64

In [71]:
df['default'].value_counts(normalize=True) * 100

default
0    83.333333
1    16.666667
Name: proportion, dtype: float64

In [72]:
#1. Setup(): Definir el dataset y parámetros
# Configuración mejorada de setup() para un dataset desbalanceado
cls = setup(
    data=df,                       # dataframe con los datos
    target='default',               # columna objetivo (clase)
    session_id=123,                 # para reproducibilidad
    normalize=True,                 # normaliza variables numéricas
    categorical_features=['estado_civil','historia_credito'],  # columnas categóricas
    numeric_imputation='mean',      # reemplazo de valores faltantes numéricos
    categorical_imputation='mode',  # reemplazo de valores faltantes categóricos
    fix_imbalance=True,             # habilita técnicas de rebalanceo de clase
    fix_imbalance_method='SMOTE',   # método para generar muestras de la clase minoritaria
    fold_strategy='stratifiedkfold' # mantiene la proporción de clases en folds de validación
)


,Description,Value
0,Session id,123
1,Target,default
2,Target type,Binary
3,Original data shape,"(150, 8)"
4,Transformed data shape,"(219, 12)"
5,Transformed train set shape,"(174, 12)"
6,Transformed test set shape,"(45, 12)"
7,Numeric features,5
8,Categorical features,2
9,Preprocess,True


In [73]:
#2. Compare_models(): Comparar multiples modelos y los rankear
best_model = compare_models(sort='F1')

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lr,Logistic Regression,0.5973,0.5715,0.4500,0.1983,0.2688,0.0443,0.0486,2.4220
ridge,Ridge Classifier,0.5882,0.5653,0.4500,0.1933,0.2640,0.0358,0.0396,0.0420
lda,Linear Discriminant Analysis,0.5882,0.5653,0.4500,0.1933,0.2640,0.0358,0.0396,0.0410
svm,SVM - Linear Kernel,0.5145,0.5382,0.5500,0.1719,0.2563,0.0057,0.0337,0.0420
knn,K Neighbors Classifier,0.3973,0.3552,0.4000,0.1220,0.1811,-0.1075,-0.1680,0.0430
qda,Quadratic Discriminant Analysis,0.7327,0.4708,0.1500,0.1333,0.1400,0.0090,0.0012,0.0530
nb,Naive Bayes,0.5773,0.3972,0.1500,0.1000,0.1167,-0.1210,-0.1468,0.0480
dt,Decision Tree Classifier,0.6827,0.4500,0.1000,0.1500,0.1167,-0.0692,-0.0710,0.0440
lightgbm,Light Gradient Boosting Machine,0.7209,0.4785,0.0500,0.0500,0.0500,-0.0872,-0.0931,0.0780
ada,Ada Boost Classifier,0.7164,0.5174,0.0500,0.0333,0.0400,-0.1017,-0.1041,0.0800


In [74]:
#3. Create model(): Crear un modelo específico
lr = create_model('lr')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7273,0.7222,0.5000,0.3333,0.4000,0.2326,0.2406
1,0.6364,0.4444,0.5000,0.2500,0.3333,0.1200,0.1336
2,0.5455,0.6667,0.5000,0.2000,0.2857,0.0351,0.0430
3,0.6364,0.7778,0.5000,0.2500,0.3333,0.1200,0.1336
4,0.7273,0.8333,0.5000,0.3333,0.4000,0.2326,0.2406
5,0.7000,0.8889,1.0000,0.2500,0.4000,0.2857,0.4082
6,0.8000,0.4444,0.0000,0.0000,0.0000,-0.1111,-0.1111
7,0.4000,0.2500,0.5000,0.1667,0.2500,-0.0714,-0.1021
8,0.3000,0.1875,0.0000,0.0000,0.0000,-0.4000,-0.5000


In [75]:
#4. Tune_model(): Ajuste automático de hiperparámetros
tuned_lr = tune_model(lr,optimize='F1')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.7273,0.7222,0.5000,0.3333,0.4000,0.2326,0.2406
1,0.7273,0.4444,0.5000,0.3333,0.4000,0.2326,0.2406
2,0.5455,0.6667,0.5000,0.2000,0.2857,0.0351,0.0430
3,0.6364,0.7778,0.5000,0.2500,0.3333,0.1200,0.1336
4,0.7273,0.8333,0.5000,0.3333,0.4000,0.2326,0.2406
5,0.7000,0.8889,1.0000,0.2500,0.4000,0.2857,0.4082
6,0.8000,0.4444,0.0000,0.0000,0.0000,-0.1111,-0.1111
7,0.4000,0.2500,0.5000,0.1667,0.2500,-0.0714,-0.1021
8,0.3000,0.1875,0.0000,0.0000,0.0000,-0.4000,-0.5000


Fitting 10 folds for each of 10 candidates, totalling 100 fits


In [ ]:
#5. plot_model(): visualizar el desempeño del modelo
evaluate_model(tuned_lr)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [77]:
#7. Predict_model(): Predicciones sobre nuevos datos en el DF
preds = predict_model(tuned_lr)
print(preds.head())

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Logistic Regression,0.6222,0.6805,0.1429,0.0833,0.1053,-0.1135,-0.1202


     edad  ingresos_mensuales estado_civil historia_credito  \
57     60                4457       Casado            Buena   
98     55                2147       Casado            Buena   
54     34                5659      Soltero          Regular   
92     64                2930       Casado            Buena   
137    33                4643       Casado            Buena   

     nro_creditos_previos  cuota_vs_ingreso  vehiculo_propio  default  \
57                      3              0.32                0        0   
98                      2              0.25                1        0   
54                      3              0.32                0        0   
92                      2              0.31                1        0   
137                     0              0.69                0        1   

     prediction_label  prediction_score  
57                  0            0.7036  
98                  0            0.8340  
54                  0            0.6995  
92            

In [78]:
#8. Save_model/load_model(): Guardar y cargar el modelo
save_model(tuned_lr,'modelo_precio_final')

Transformation Pipeline and Model Successfully Saved


(Pipeline(memory=Memory(location=None),
          steps=[('numerical_imputer',
                  TransformerWrapper(exclude=None,
                                     include=['edad', 'ingresos_mensuales',
                                              'nro_creditos_previos',
                                              'cuota_vs_ingreso',
                                              'vehiculo_propio'],
                                     transformer=SimpleImputer(add_indicator=False,
                                                               copy=True,
                                                               fill_value=None,
                                                               keep_empty_features=False,
                                                               missing_values=nan,
                                                               strategy='mean'))),
                 ('categorical_impu...
                  TransformerWrapper(exclude=None, include=

### 📊 CONCLUSIONES MODELO DE CLASIFICACIÓN

#### 1. Desempeño General

* El modelo seleccionado fue **Regresión Logística**, tras comparar múltiples clasificadores (SVM, KNN, QDA, Naive Bayes, Árboles, Boosting, Random Forest, etc.).
* En promedio, el modelo alcanzó una **exactitud (Accuracy) del 60.6%**, con una **AUC de 0.57**, lo cual indica una capacidad **moderada de discriminación** entre clases.

#### 2. Métricas Principales

* **Recall promedio:** 0.45 → el modelo logra identificar correctamente el 45% de los casos positivos.
* **Precisión promedio:** 0.21 → de las predicciones positivas, solo el 21% son correctas.
* **F1-score:** 0.28 → balance moderado entre precisión y exhaustividad.
* **Kappa (0.055) y MCC (0.059):** valores bajos, sugiriendo una correlación limitada entre predicciones y etiquetas reales.
* La **variabilidad entre folds** (std ≈ 0.15 en Accuracy y 0.23 en AUC) refleja cierta **inestabilidad del modelo**, probablemente por tamaño muestral limitado o desbalance de clases.

#### 3. Evaluación Comparativa

* Aunque otros modelos (como QDA, Extra Trees o LightGBM) mostraron mayores *accuracy* en validación cruzada simple, sus métricas de *recall* y *F1* fueron nulas o muy bajas, indicando **overfitting o sesgo hacia una clase mayoritaria**.
* La **Regresión Logística** mantiene un **equilibrio razonable** entre las métricas, mostrando **robustez** frente a la variabilidad de los datos.

#### 4. Interpretación y Aspectos Técnicos

* La **AUC cercana a 0.57** sugiere que el modelo tiene una **capacidad limitada para ordenar correctamente** instancias positivas y negativas.
* El comportamiento de métricas asimétricas (Precision–Recall) podría indicar **desequilibrio de clases** o **valores atípicos** en la variable respuesta.
* Las iteraciones con **recall = 1 y precisión baja (0.25)** muestran que en ciertos folds el modelo prefirió **minimizar falsos negativos** a costa de cometer más falsos positivos.

#### 5. Recomendaciones

* Aplicar **reescalado y balanceo de clases** (SMOTE, undersampling o ponderación de clases).
* Evaluar **regularización L1 o Elastic Net** para estabilizar coeficientes y reducir varianza.
* Analizar **importancia de variables** mediante los coeficientes del modelo y eliminar predictores irrelevantes.
* Considerar modelos no lineales (árboles o SVM con kernel) solo tras asegurar un adecuado balance de clases.

---

#### 🧩 CONCLUSIÓN GLOBAL

> El modelo de clasificación basado en **Regresión Logística** presenta un **ajuste moderado** y un desempeño **aceptable considerando la complejidad del problema y el desbalance de clases**.
> Si bien la capacidad predictiva (AUC ≈ 0.57) no es alta, el modelo ofrece **interpretabilidad, estabilidad y una base sólida** para mejorar mediante técnicas de regularización y balanceo.

